# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 12 2022 8:30AM,244011,10,CTF0011361,"Citieffe, Inc.",Released
1,Aug 12 2022 8:30AM,244012,10,0085935695,ISDIN Corporation,Released
2,Aug 12 2022 8:30AM,244012,10,0085935700,ISDIN Corporation,Released
3,Aug 12 2022 8:30AM,244012,10,0085935701,ISDIN Corporation,Released
4,Aug 12 2022 8:30AM,244012,10,0085935711,ISDIN Corporation,Released
5,Aug 12 2022 8:30AM,244012,10,0085935713,ISDIN Corporation,Released
6,Aug 12 2022 8:30AM,244012,10,0085935731,ISDIN Corporation,Released
7,Aug 12 2022 8:30AM,244012,10,0085935730,ISDIN Corporation,Released
8,Aug 12 2022 8:30AM,244012,10,0085935733,ISDIN Corporation,Released
9,Aug 12 2022 8:30AM,244012,10,0085935735,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,244012,Released,12
22,244014,Released,1
23,244015,Released,1
24,244016,Released,1
25,244017,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
244012,NaN,NaN,NaN,12.0
244014,NaN,NaN,NaN,1.0
244015,NaN,NaN,NaN,1.0
244016,NaN,NaN,NaN,1.0
244017,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
243942,0.0,0.0,0.0,43.0
243961,0.0,1.0,9.0,9.0
243963,1.0,0.0,0.0,17.0
243968,0.0,0.0,0.0,1.0
243969,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
243942,0.0,0,0,43
243961,0.0,1,9,9
243963,1.0,0,0,17
243968,0.0,0,0,1
243969,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,243942,0.0,0,0,43
1,243961,0.0,1,9,9
2,243963,1.0,0,0,17
3,243968,0.0,0,0,1
4,243969,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,243942,0.0,,,43
1,243961,0.0,1,9,9
2,243963,1.0,,,17
3,243968,0.0,,,1
4,243969,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released       int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 12 2022 8:30AM,244011,10,"Citieffe, Inc."
1,Aug 12 2022 8:30AM,244012,10,ISDIN Corporation
13,Aug 11 2022 6:44PM,244017,21,"NBTY Global, Inc."
14,Aug 11 2022 4:56PM,244016,16,"SHL Pharma, LLC"
15,Aug 11 2022 4:51PM,244015,16,"Emersa Waterbox, LLC"
16,Aug 11 2022 4:35PM,244014,102,"Senseonics, Incorporated"
17,Aug 11 2022 4:00PM,244010,19,ACG North America LLC
18,Aug 11 2022 3:21PM,244001,16,Seqirus
19,Aug 11 2022 3:19PM,244000,22,"NBTY Global, Inc."
20,Aug 11 2022 3:05PM,243995,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Aug 12 2022 8:30AM,244011,10,"Citieffe, Inc.",0.0,,,1
1,Aug 12 2022 8:30AM,244012,10,ISDIN Corporation,0.0,,,12
2,Aug 11 2022 6:44PM,244017,21,"NBTY Global, Inc.",0.0,,,1
3,Aug 11 2022 4:56PM,244016,16,"SHL Pharma, LLC",0.0,,,1
4,Aug 11 2022 4:51PM,244015,16,"Emersa Waterbox, LLC",0.0,,,1
5,Aug 11 2022 4:35PM,244014,102,"Senseonics, Incorporated",0.0,,,1
6,Aug 11 2022 4:00PM,244010,19,ACG North America LLC,0.0,,,1
7,Aug 11 2022 3:21PM,244001,16,Seqirus,0.0,,,1
8,Aug 11 2022 3:19PM,244000,22,"NBTY Global, Inc.",0.0,,,1
9,Aug 11 2022 3:05PM,243995,19,ACG North America LLC,0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 8:30AM,244011,10,"Citieffe, Inc.",1,,
1,Aug 12 2022 8:30AM,244012,10,ISDIN Corporation,12,,
2,Aug 11 2022 6:44PM,244017,21,"NBTY Global, Inc.",1,,
3,Aug 11 2022 4:56PM,244016,16,"SHL Pharma, LLC",1,,
4,Aug 11 2022 4:51PM,244015,16,"Emersa Waterbox, LLC",1,,
5,Aug 11 2022 4:35PM,244014,102,"Senseonics, Incorporated",1,,
6,Aug 11 2022 4:00PM,244010,19,ACG North America LLC,1,,
7,Aug 11 2022 3:21PM,244001,16,Seqirus,1,,
8,Aug 11 2022 3:19PM,244000,22,"NBTY Global, Inc.",1,,
9,Aug 11 2022 3:05PM,243995,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 8:30AM,244011,10,"Citieffe, Inc.",1,,
1,Aug 12 2022 8:30AM,244012,10,ISDIN Corporation,12,,
2,Aug 11 2022 6:44PM,244017,21,"NBTY Global, Inc.",1,,
3,Aug 11 2022 4:56PM,244016,16,"SHL Pharma, LLC",1,,
4,Aug 11 2022 4:51PM,244015,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 8:30AM,244011,10,"Citieffe, Inc.",1,NaN,NaN
1,Aug 12 2022 8:30AM,244012,10,ISDIN Corporation,12,NaN,NaN
2,Aug 11 2022 6:44PM,244017,21,"NBTY Global, Inc.",1,NaN,NaN
3,Aug 11 2022 4:56PM,244016,16,"SHL Pharma, LLC",1,NaN,NaN
4,Aug 11 2022 4:51PM,244015,16,"Emersa Waterbox, LLC",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 8:30AM,244011,10,"Citieffe, Inc.",1,0.0,0.0
1,Aug 12 2022 8:30AM,244012,10,ISDIN Corporation,12,0.0,0.0
2,Aug 11 2022 6:44PM,244017,21,"NBTY Global, Inc.",1,0.0,0.0
3,Aug 11 2022 4:56PM,244016,16,"SHL Pharma, LLC",1,0.0,0.0
4,Aug 11 2022 4:51PM,244015,16,"Emersa Waterbox, LLC",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2439788,79,1.0,12.0
102,244014,1,0.0,0.0
16,732032,3,0.0,0.0
19,731990,3,0.0,0.0
20,487924,26,9.0,1.0
21,244017,1,0.0,0.0
22,244000,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2439788,79,1.0,12.0
1,102,244014,1,0.0,0.0
2,16,732032,3,0.0,0.0
3,19,731990,3,0.0,0.0
4,20,487924,26,9.0,1.0
5,21,244017,1,0.0,0.0
6,22,244000,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,79,1.0,12.0
1,102,1,0.0,0.0
2,16,3,0.0,0.0
3,19,3,0.0,0.0
4,20,26,9.0,1.0
5,21,1,0.0,0.0
6,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,102,Released,1.0
2,16,Released,3.0
3,19,Released,3.0
4,20,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,19,20,21,22
Status,,,,,,,
Completed,12.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,9.0,0.0,0.0
Released,79.0,1.0,3.0,3.0,26.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,19,20,21,22
0,Completed,12.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,9.0,0.0,0.0
2,Released,79.0,1.0,3.0,3.0,26.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,19,20,21,22
0,Completed,12.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,9.0,0.0,0.0
2,Released,79.0,1.0,3.0,3.0,26.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()